**DEMO Visualization with Altair - Gender Gap**
---
*Develop in Google Colab*

# DATOS

In [ ]:
import pandas as pd

# obtener archivos fuente almacenados en google drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/My Drive/Master/Visualización/Gender_StatsData.csv',delimiter=',')
df.head(2)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,Unnamed: 63
0,Arab World,ARB,"Access to anti-retroviral drugs, female (%)",SH.HIV.ARTC.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arab World,ARB,"Access to anti-retroviral drugs, male (%)",SH.HIV.ARTC.MA.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# transformación de la data para cambiar las columnas de años por registros en filas (también eliminación de NaN)
col_names = df.columns[4:-1]
df = pd.melt(df, id_vars=['Country Name','Country Code','Indicator Name','Indicator Code'], value_vars=col_names).dropna()
df.columns = ['country', 'country_code','indicator','indicator_code','year','value']

In [ ]:
df.head()

,country,country_code,indicator,indicator_code,year,value
6,Arab World,ARB,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,134.214763
9,Arab World,ARB,Age dependency ratio (% of working-age populat...,SP.POP.DPND,1960,88.210387
22,Arab World,ARB,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,1960,47.694589
48,Arab World,ARB,"Death rate, crude (per 1,000 people)",SP.DYN.CDRT.IN,1960,19.872061
116,Arab World,ARB,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,1960,6.950147


In [ ]:
# dataset contries
countries = pd.read_csv('/content/drive/My Drive/Master/Visualización/Gender_StatsCountry.csv',delimiter=',')
countries = countries[['Country Code','2-alpha code','Region','Income Group']]
countries.columns = ['country_code','country_codeG','region', 'income_group']
countries.head(2)

,country_code,country_codeG,region,income_group
0,ABW,AW,Latin America & Caribbean,High income
1,AFG,AF,South Asia,Low income


In [ ]:
# importación de gapminder, al cuál enlazaré por nombre de país
from vega_datasets import data
gapminder = data.gapminder.url
gapminder

'https://vega.github.io/vega-datasets/data/gapminder.json'

Trabajo con dos datasets:
*   Data : contiene información de los indicadores por años.
*   Countries : contiene características de los países.

# VISUALIZACIONES

In [ ]:
import altair as alt
import numpy as np

# deshabilito para solucionar error por filas máximas
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [ ]:
# listado de los indicadores presentes en el dataset
indicators = pd.DataFrame(df,columns=['indicator_code','indicator'])
indicators = indicators.drop_duplicates()
indicators.sort_values(by=['indicator']).head(5)

,indicator_code,indicator
5808696,SH.HIV.ARTC.FE.ZS,"Access to anti-retroviral drugs, female (%)"
5808697,SH.HIV.ARTC.MA.ZS,"Access to anti-retroviral drugs, male (%)"
7403978,FX.OWN.TOTL.FE.ZS,Account ownership at a financial institution o...
7403979,FX.OWN.TOTL.MA.ZS,Account ownership at a financial institution o...
1451764,SE.PRM.TENR.FE,"Adjusted net enrollment rate, primary, female ..."


**- #1 Indicador analizado:**
Proportion of time spent on unpaid domestic and care work (% of 24 hour day)

In [ ]:
# filtro la data según los indicadores para cada género. 
# realizo el filtrado antes y no en el gráfico, ya que el tiempo de ejecución es más rápido.

## indicador femenino
is_SGTIMUWRKFE = df['indicator_code']=='SG.TIM.UWRK.FE'
data_SGTIMUWRKFE = df[is_SGTIMUWRKFE]

## indicador masculino
is_SGTIMUWRKMA = df['indicator_code']=='SG.TIM.UWRK.MA'
data_SGTIMUWRKMA = df[is_SGTIMUWRKMA]

data_SGTIMUWRKFE['gender'] = 'Femenino'
data_SGTIMUWRKMA['gender'] = 'Masculino'

data_SGTIMUWRK = pd.concat([data_SGTIMUWRKFE,data_SGTIMUWRKMA], ignore_index=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [ ]:
# analizo los registros por región, considerando el valor máximo por año.
ch1 = alt.Chart(data_SGTIMUWRK).mark_line().encode(
        x=alt.X('year:T', axis=alt.Axis(labelAlign='left'), title='Año'),
        y=alt.Y('max(value):Q', title='Valor de Indicador'),
        color=alt.Color('region:N',title='Región'),
        column=alt.Column('gender:N', title='Género')
    ).transform_lookup(
      lookup='country_code',
      from_=alt.LookupData(countries, 'country_code', ['region'])).properties(title='Tiempo invertido en tareas domésticas por género')
ch1

alt.Chart(...)

In [ ]:
# para una mejor comparación, analizo el indicador con los valores promedio según género por año (algo más general)
ch2 = alt.Chart(data_SGTIMUWRK).mark_line(strokeDash=[4,4],strokeWidth=3).encode(
        x=alt.X('year:T', axis=alt.Axis(labelAlign='left'), title='Año'),
        y=alt.Y('average(value):Q', title='Valor de Indicador'),
        color=alt.Color('gender:N',title='Género',scale=alt.Scale(domain=['Femenino', 'Masculino'],range=['pink', 'blue']))
    )
ch2

alt.Chart(...)

In [ ]:
# para el primer gráfico agrego un selector por región para visualizar mejor los cambios 
input_dropdown = alt.binding_select(
    options = ['East Asia & Pacific', 'Europe & Central Asia', 'Latin America & Caribbean',
               'Middle East & North Africa', 'North America', 'South Asia', 'Sub-Saharan Africa']
)

dropSelect = alt.selection_single(fields=['region'],
                                  bind=input_dropdown,
                                  name='Region')

ch3 = alt.Chart(data_SGTIMUWRK).mark_line().encode(
        x=alt.X('year:T', axis=alt.Axis(labelAlign='left'), title='Año'),
        y=alt.Y('average(value):Q', title='Valor de Indicador'),
        color=alt.condition(dropSelect, 'region:N',
                        alt.value('lightgray')),
        column=alt.Column('gender:N', title='Género')
    ).transform_lookup(
      lookup='country_code',
      from_=alt.LookupData(countries, 'country_code', ['region'])).add_selection(dropSelect)
ch3

alt.Chart(...)

In [ ]:
# para visualizar en el mapa, trabajo con una fuente diferente a la de Vegas, ya que no tenía datos para enlazar. Con esta fuente enlazo por el nombre.
world = alt.topo_feature('https://cdn.jsdelivr.net/npm/world-atlas@2/countries-110m.json','countries')

# género: femenino
ch4 = alt.Chart(world).mark_geoshape().encode(
    color='value:Q'
).transform_lookup(
    lookup='properties.name',
    from_=alt.LookupData(data_SGTIMUWRKFE, 'country', ['value'])
).properties(
    projection={'type':'equirectangular'},
    width=500, height=300
)
ch4

alt.Chart(...)

In [ ]:
# género: masculino
ch5 = alt.Chart(world).mark_geoshape().encode(
    color='value:Q'
).transform_lookup(
    lookup='properties.name',
    from_=alt.LookupData(data_SGTIMUWRKMA, 'country', ['value'])
).properties(
    projection={'type':'equirectangular'},
    width=500, height=300
)
ch5

alt.Chart(...)

**#1**

**Significado:**
El tiempo promedio que las mujeres / los hombres dedican a la provisión de servicios domésticos para consumo propio. Los datos se expresan como una proporción de tiempo en un día. 

**Interpretación de indicador:**
Observo que las mujeres son aquellas que dedican mayor tiempo en promedio a los servicios domésticos.

**- #2 Indicadores analizados:**

. Women are able to work in the same industries as men (1=yes; 0=no)

. Nonpregnant and nonnursing women can do the same jobs as men (1=yes; 0=no)

In [ ]:
# filtro indicadores
is_SGINDWORKEQ = df['indicator_code']=='SG.IND.WORK.EQ'
data_SGINDWORKEQ = df[is_SGINDWORKEQ]

is_SGJOBNOPNEQ = df['indicator_code']=='SG.JOB.NOPN.EQ'
data_SGJOBNOPNEQ = df[is_SGJOBNOPNEQ]

In [ ]:
# primer indicador : muestro datos en gráfico de barras por región
ch6 = alt.Chart(data_SGINDWORKEQ).mark_bar().encode(
    alt.X('count():Q', axis= alt.Axis(gridDash=[4,3])),
    y='region:N',
    color=alt.Color('value:N', scale=alt.Scale(
            domain=['0', '1'],
            range=['blue', 'pink']))
).transform_lookup(
      lookup='country_code',
      from_=alt.LookupData(countries, 'country_code', ['region']))
ch6

alt.Chart(...)

In [ ]:
# segundo indicador: gráfico de barras por región
ch7 = alt.Chart(data_SGJOBNOPNEQ).mark_bar().encode(
    alt.X('count():Q', axis= alt.Axis(gridDash=[4,3])),
    y='region:N',
    color=alt.Color('value:N', scale=alt.Scale(
            domain=['0', '1'],
            range=['blue', 'pink']))
).transform_lookup(
      lookup='country_code',
      from_=alt.LookupData(countries, 'country_code', ['region']))

ch7

alt.Chart(...)

In [ ]:
# primer indicador: muestro los datos en un mapa, activando los colores según el valor (0, 1)
# existen registros 'null' (podrían perderse algunos valores en el join por nombre), los muestro como grises
ch8 = alt.Chart(world).mark_geoshape().encode(
    color=alt.Color('value:N', scale=alt.Scale(
            domain=['null','0', '1'],
            range=['gray','blue', 'pink']))
).transform_lookup(
    lookup='properties.name',
    from_=alt.LookupData(data_SGINDWORKEQ, 'country', ['value'])
).properties(
    projection={'type':'equirectangular'},
    width=500, height=300
).properties(title='Países donde se considera a la mujer tan capaz como al hombre')
ch8

alt.Chart(...)

In [ ]:
# segundo indicador: muestro los datos en un mapa, activando los colores según el valor (0, 1)
# existen registros 'null' (podrían perderse algunos valores en el join por nombre), los muestro como grises
ch9 = alt.Chart(world).mark_geoshape().encode(
     color=alt.Color('value:N', scale=alt.Scale(
            domain=['null','0', '1'],
            range=['gray','blue', 'pink']))
).transform_lookup(
    lookup='properties.name',
    from_=alt.LookupData(data_SGJOBNOPNEQ, 'country', ['value'])
).properties(
    projection={'type':'equirectangular'},
    width=500, height=300
).properties(title='Países donde se considera que la mujer no embarazada puede hacer las mismas tareas que un hombre')
ch9

alt.Chart(...)

**#2**

**Significado:** 
Si las mujeres son capaces de trabajar en las mismas industrias que los hombres y si las mujeres que no están embarazadas pueden hacer el mismo trabajo que los hombres.

**Interpretación de indicadores:** Existen regiones en las que se piensa en más del 50% de países, que las mujeres no son capaces de hacer el mismo trabajo que los hombres, por ejemplo el norte y sur de África.

**Colores:** En los gráficos de mapas, tengo como objetivo mostrar cuando los indicadores estan activados un color rosa y cuando no en azul. Como ejemplo: cuando no hay brecha de género el color es el rosa.

**- #3 Indicador analizado:** Female professional and technical workers (% of total)

In [ ]:
# filtro por indicador
is_SGGENTECHZS = df['indicator_code']=='SG.GEN.TECH.ZS'
data_SGGENTECHZS = df[is_SGGENTECHZS]

In [ ]:
# para este gráfico cruzo la data con el dataset 'gapminder' que contiene la población y la tasa de fertilidad, con fin de mostrar un gráfico de burbujas
ch10 = alt.Chart(data_SGGENTECHZS).mark_point().encode(
    x=alt.X('fertility:Q', title='Tasa de Fertilidad'),
    y=alt.Y('value:Q',title='Valor de Indicador'),
    size=alt.Size('pop:Q', title='Población'),
    tooltip=['country:N','value:Q', 'fertility:Q']
).transform_lookup(
      lookup='country',
      from_=alt.LookupData(gapminder, 'country', ['fertility','pop']))
ch10

alt.Chart(...)

In [ ]:
# para mostrar la data y filtrarla por región, decido hacer un join para obtener este campo.
merged_SGGENTECHZS = data_SGGENTECHZS.join(countries.set_index('country_code'), on='country_code')

# agrego al gráfico anterior un selector
ch11 = alt.Chart(merged_SGGENTECHZS).mark_point().encode(
    x=alt.X('fertility:Q', title='Tasa de Fertilidad'),
    y=alt.Y('value:Q',title='Valor de Indicador'),
    size=alt.Size('pop:Q', title='Población'),
    tooltip=['country:N','value:Q', 'fertility:Q'],
    color=alt.condition(dropSelect, 'region:N',alt.value('lightgray'), title='Región'),
).transform_lookup(
      lookup='country',
      from_=alt.LookupData(gapminder, 'country', ['fertility','pop'])).add_selection(dropSelect).properties(title='% de profesionales mujeres por país/región (tasa de fertilidad/población)')
ch11

alt.Chart(...)

**Interpretación :** La mayoría de países con menor tasa de fertilidad son aquellos que tienen mayor porcentaje de mujeres profesionales.


# VISUALIZACIÓN FINAL - DASHBOARD

Analicé los indicadores de la categoría 'Gender: Participation & access' con la finalidad de mostrar cómo se encuentra el reconocimiento del  género femenino a nivel mundial.
Los indicadores son:
- Female professional and technical workers (% of total)
- Women are able to work in the same industries as men (1=yes; 0=no)
- Nonpregnant and nonnursing women can do the same jobs as men (1=yes; 0=no)
- Proportion of time spent on unpaid domestic and care work, female (% of 24 hour day)
- Proportion of time spent on unpaid domestic and care work, male (% of 24 hour day)

In [ ]:
print("DASHBOARD FINAL - GENDER GAP")
ch1

DASHBOARD FINAL - GENDER GAP


alt.Chart(...)

In [ ]:
ch8.properties(width=450)|ch9.properties(width=450)

alt.HConcatChart(...)

In [ ]:
ch11

alt.Chart(...)